In [1]:
# !pip install pandas
# !pip install dash
# !pip install jupyter-dash

In [2]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from dash import Dash, dcc, html, Input, Output, State, no_update


In [3]:
def extrair_ano(cat_number):
    return int('20' + cat_number.split('-')[1][:2])

def criar_df_area_empilhada(df,taxon,ano='year'):
    df_area = df.copy()
    df_area = df_area.groupby([taxon,ano]).size().reset_index(name='counts')
    df_area = df_area.sort_values(by=ano)
    df_area[ano] = pd.to_numeric(df_area[ano])
    df_area = df_area.pivot_table(index=taxon, columns=ano, values='counts', fill_value=0)
    df_area = df_area.cumsum(axis=1)
    df_area = df_area.reset_index()

    df_area_long = pd.melt(df_area, id_vars=[taxon], var_name=ano, value_name='value')
    last_year_order_counts = df_area_long[df_area_long[ano] == df_area_long[ano].max()].groupby(taxon).size()
    df_area_long_sorted = df_area_long.set_index(taxon).loc[last_year_order_counts.index].reset_index()

    sorted_order = df_area_long_sorted[df_area_long_sorted[ano] == max(df_area_long_sorted[ano])].groupby(taxon)['value'].sum().sort_values(ascending=False).index

    return df_area_long_sorted,sorted_order

def criar_df_linha(df,taxon,ano='year'):
    df_linha = df.copy()
    df_linha = df_linha.groupby([taxon, ano]).size().reset_index(name='counts')
    df_linha = df_linha.sort_values(by=ano)
    df_linha[ano] = pd.to_numeric(df_linha[ano])
    df_linha = df_linha.pivot_table(index=taxon, columns=ano, values='counts', fill_value=0)
    df_linha_soma = df_linha.cumsum(axis=1)
    df_linha_soma = df_linha_soma.reset_index()
    df_linha = df_linha.reset_index()

    sorted_order = list_sorted_order(df_linha_soma,taxon,ano)

    df_linha[taxon] = pd.Categorical(df_linha[taxon], categories=sorted_order, ordered=True)
    df_linha = df_linha.sort_values(by=taxon)
    return df_linha,sorted_order

def list_sorted_order(df,taxon,ano):
    df_area_long = pd.melt(df, id_vars=[taxon], var_name=ano, value_name='value')
    last_year_order_counts = df_area_long[df_area_long[ano] == df_area_long[ano].max()].groupby(taxon).size()
    df_area_long_sorted = df_area_long.set_index(taxon).loc[last_year_order_counts.index].reset_index()

    sorted_order = df_area_long_sorted[df_area_long_sorted[ano] == max(df_area_long_sorted[ano])].groupby(taxon)['value'].sum().sort_values(ascending=False).index
    return sorted_order

def calcula_condicional_ausentes_porcentagem(df, previous_level, current_level):
    conditioned_df = df[df[previous_level].notnull()]
    return (conditioned_df[current_level].isnull().sum() / len(conditioned_df)) * 100


In [4]:
df = pd.read_csv('planilha_unificada.csv',low_memory=False)

df['specie'] = df.apply(lambda row: row['genus'] + ' ' + row['specificEpithet'] if not pd.isna(row['genus']) and not pd.isna(row['specificEpithet']) else None, axis=1)

df['year'] = df['eventDate'].str.split('/').str[0].str.split('-').str[0]
df['mounth'] = df['eventDate'].str.split('/').str[0].str.split('-').str[1]
df['day'] = df['eventDate'].str.split('/').str[0].str.split('-').str[2]
df['ano'] = df['catalogNumber'].apply(lambda x: extrair_ano(x) if isinstance(x, str) else None)

In [5]:
# Filtrando as colunas sem informações
col = ['continent', 'country','stateProvince','county','locality']
columns_to_fill = ['kingdom', 'phylum', 'class', 'order', 'superfamily', 'family', 'subfamily', 'genus', 'specie']
df_sem_classificao = df.assign(**{col: df[col].fillna('sem classificação') for col in columns_to_fill})
df_treemap_classificacao = df.assign(**{col: df[col].fillna('sem classificação') for col in columns_to_fill})

df_paises = df.assign(**{col: df[col].fillna('sem informação do local') for col in col})
df_sem_brasil = df_paises[df_paises['country'] != 'Brasil']
df_brasil = df_paises[df_paises['country'] == 'Brasil']
df_sem_mg = df_brasil[df_brasil['stateProvince'] != 'Minas Gerais']

In [6]:
# COLOCAR O DF INTEIRO (TALVEZ COLOCAR ALGUNS FILTROS)

In [7]:

treemap_class_taxon = px.treemap(df_treemap_classificacao, path=['kingdom','phylum', 'class','order','superfamily','family','subfamily','genus','specie'],color='order')
treemap_class_taxon.update_layout(margin = dict(t=50, l=25, r=25, b=25))

treemap_paises = px.treemap(df_paises, path=[px.Constant("world"), 'continent', 'country','stateProvince','county','locality'])
treemap_paises.update_layout(margin = dict(t=50, l=25, r=25, b=25))

treemap_sem_brasil = px.treemap(df_sem_brasil, path=[px.Constant("world"), 'continent', 'country','stateProvince','county','locality'])

treemap_brasil = px.treemap(df_brasil, path=['country','stateProvince','county','locality'])

treemap_sem_mg = px.treemap(df_sem_mg, path=['country','stateProvince','county','locality'])


In [8]:
tax_levels = ['order', 'superfamily', 'family', 'subfamily', 'genus','subgenus','specie']
area_empilhada_dict = {}
sorted_orders = {}

for tax in tax_levels:
    # DataFrame com a coluna 'ano'
    df_tax_ano, sorted_tax_ano = criar_df_area_empilhada(df_sem_classificao, tax, 'ano')
    area_empilhada_ano = px.area(df_tax_ano, x='ano', y='value', color=tax,
                      title=f'Contagem de Coletas por {tax.capitalize()} (Empilhado por Ano)',
                      labels={'value': 'Contagem de Coletas', tax: tax.capitalize(), 'ano': 'Ano'},
                      category_orders={tax: sorted_tax_ano})

    # DataFrame com a coluna 'year'
    df_tax_year, sorted_tax_year = criar_df_area_empilhada(df_sem_classificao, tax, 'year')
    area_empilhada_year = px.area(df_tax_year, x='year', y='value', color=tax,
                       title=f'Contagem de Coletas por {tax.capitalize()} (Empilhado por Ano)',
                       labels={'value': 'Contagem de Coletas', tax: tax.capitalize(), 'year': 'Ano'},
                       category_orders={tax: sorted_tax_year})

    area_empilhada_dict[tax] = {'ano': area_empilhada_ano, 'year': area_empilhada_year}
    sorted_orders[tax] = sorted_tax_ano

# Criar uma figura de plotly.graph_objects
grafico_area_empilhada = go.Figure()

# categories = []
for tax in tax_levels:
    for trace in area_empilhada_dict[tax]['ano'].data:
        if trace.name == 'sem classificação':
            trace.visible = 'legendonly'
        grafico_area_empilhada.add_trace(trace)
        grafico_area_empilhada.data[-1].visible = False  # Inicialmente definir visível como False

    for trace in area_empilhada_dict[tax]['year'].data:
        grafico_area_empilhada.add_trace(trace)
        grafico_area_empilhada.data[-1].visible = False  # Inicialmente definir visível como False
        # categories.append((tax, 'year'))

# Definir a visibilidade inicial (genus e ano)
for i in range(len(area_empilhada_dict['order']['ano'].data)):
    grafico_area_empilhada.data[i].visible = True



In [9]:
histograma_tax_dict = {}
for tax in tax_levels:
    df_tax_filtered = df_sem_classificao[df_sem_classificao[tax] != 'sem classificação']
    # df_tax_filtered[tax] = df_tax_filtered[tax].apply(lambda x: [s.strip() for s in x.split(' | ')] if isinstance(x, str) else x)

    # # Explodir a lista resultante em várias linhas e criar uma nova coluna
    # df_tax_filtered = df_tax_filtered.explode(tax).reset_index(drop=True)
    # DataFrame com a contagem dos itens classificados
    df_tax_count = df_tax_filtered[tax].value_counts()
    hist_tax = px.bar(df_tax_count,log_y=True)
    hist_tax.update_layout(
        xaxis = dict(
            tickmode = 'array',
            tickvals = df_tax_count.index,  # Define os valores dos rótulos
            ticktext = [str(label)[:15] for label in df_tax_count.index]  # Limita o comprimento dos rótulos
        )
    )
    histograma_tax_dict[tax] = hist_tax


In [10]:
# df_ano_corri = df[df['year'].str.len() == 4]
# df_ano_corri = df_ano_corri[df_ano_corri['year']!='2061']
# Fazer um de linha para as coletas (só tem pra identificacao)
linha_tax_dict = {}

for tax in tax_levels:
    df_linha, sorted_orders = criar_df_linha(df_sem_classificao, tax)

    todos_anos = range(max(1984, df_linha.iloc[:, 1:].values.min()), df_linha.columns[1:].max() + 1)
    anos_faltantes = [ano for ano in todos_anos if ano not in df_linha.columns[1:]]
    
    traces=[]
    linha_tax_unico_dict = {}  # Dicionário interno para armazenar os gráficos de linha por categoria
    for index, row in df_linha.iterrows():
        visibility = 'legendonly' if row[tax] == 'sem classificação' else True
        trace = go.Scatter(x=df_linha.columns[1:], y=row.values[1:], mode='lines+markers', name=row[tax], visible=visibility)
        traces.append(trace)
        grafico_linha_unico = px.line(row,markers=True)
        linha_tax_unico_dict[row[tax]] = grafico_linha_unico

    layout = go.Layout(
        xaxis=dict(title='Ano'),
        yaxis=dict(
            title='Contagem de Coletas',
            range=[0 - (df_linha.iloc[:, 1:].values.max() * 0.03), df_linha.iloc[:, 1:].values.max() * 1.03]
        )
    )

    grafico_linha_todos = go.Figure(data=traces, layout=layout)
    for index, valor in enumerate(anos_faltantes):
        grafico_linha_todos.add_shape(
            type="rect",
            x0=valor,
            y0=0,
            x1=valor,
            y1=df_linha.iloc[:, 1:].values.max() * 100,  # Altura máxima com base nos dados
            opacity=0.3,
            line=dict(color="Gray", width=10)
        )

    grafico_linha_todos.update_layout(xaxis=dict(showgrid=False))
    linha_tax_unico_dict['all'] = grafico_linha_todos
    linha_tax_dict[tax] = linha_tax_unico_dict


In [11]:

# linha_tax_dict['family']['all']


In [12]:
df_percent_class = df[['order','Coorte ou Supercoorte','superfamily','family','subfamily','genus','subgenus','specie']]

valores_null_porcentagem = (df_percent_class.isnull().sum() / len(df_percent_class)) * 100

grafico_valores_null = px.bar(valores_null_porcentagem)


In [13]:

# Lista de níveis hierárquicos na ordem desejada
taxon_levels = ['class','order', 'Coorte ou Supercoorte','superfamily', 'family','genus', 'specie']

# Dicionário para armazenar as porcentagens
porcentagem_valores_ausentes = {}

# Calcular as porcentagens para cada nível hierárquico
for i in range(1, len(taxon_levels)):
    nivel_anterior = taxon_levels[i - 1]
    nivel_atual = taxon_levels[i]
    porcentagem_valores_ausentes[nivel_atual] = calcula_condicional_ausentes_porcentagem(df, nivel_anterior, nivel_atual)

# Criar um DataFrame a partir do dicionário de porcentagens
df_porcentagem_valores_ausentes = pd.DataFrame(list(porcentagem_valores_ausentes.items()), columns=['nivel', 'valor'])

grafico_valores_null_dependente = px.bar(df_porcentagem_valores_ausentes,x='nivel',y='valor')


In [14]:
df_tax_unicos = df[tax_levels].nunique()

grafico_unicos_taxon = px.bar(df_tax_unicos)


In [15]:
barras_coletores_dict = {}
df_coletores = df.copy()
df_coletores = df_coletores[['recordedBy','year','ano']]

df_coletores['recordedBy'] = df_coletores['recordedBy'].dropna().apply(lambda x: [s.strip() for s in x.split(' | ')] if isinstance(x, str) else x)

# Explodir a lista resultante em várias linhas e criar uma nova coluna
df_coletores = df_coletores.explode('recordedBy').reset_index(drop=True)

df_coletores_grouped_coleta = df_coletores.groupby('year')['recordedBy'].nunique().reset_index()
df_coletores_grouped_identificacao = df_coletores.groupby('ano')['recordedBy'].nunique().reset_index()

# Criar o gráfico de barras
grafico_barras_coletores_coleta = px.bar(df_coletores_grouped_coleta,x='year',y='recordedBy')
barras_coletores_dict['year'] = grafico_barras_coletores_coleta

grafico_barras_coletores_identificacao = px.bar(df_coletores_grouped_identificacao,x='ano',y='recordedBy')
barras_coletores_dict['ano'] = grafico_barras_coletores_identificacao


In [16]:
# Verificar os dados de cada um dos coletores
barras_equipe_dict = {}
df_filtrado_equipe = df_coletores[df_coletores['recordedBy'].str.contains('Equipe', na=False)]

# Contar a quantidade de ocorrências em cada ano
df_contagem_coleta = df_filtrado_equipe.groupby(['year', 'recordedBy']).size().reset_index(name='count')
df_contagem_identificacao = df_filtrado_equipe.groupby(['ano', 'recordedBy']).size().reset_index(name='count')

grafico_barras_equipe_coleta = px.bar(df_contagem_coleta, x='year', y='count', color='recordedBy', barmode='stack',
             labels={'year': 'Ano', 'count': 'Contagem', 'recordedBy': 'Registrado por'},
             title='Contagem de ocorrências por ano e registrado por')
barras_equipe_dict['year'] = grafico_barras_equipe_coleta

grafico_barras_equipe_identificacao = px.bar(df_contagem_identificacao, x='ano', y='count', color='recordedBy', barmode='stack',
             labels={'year': 'Ano', 'count': 'Contagem', 'recordedBy': 'Registrado por'},
             title='Contagem de ocorrências por ano e registrado por')
barras_equipe_dict['ano'] = grafico_barras_equipe_identificacao


In [17]:

barras_paises_dict = {}
df_ano = df.copy()
df_agrupado_cole = df_ano.groupby(['country', 'year']).size().reset_index(name='counts')
df_agrupado = df_ano.groupby(['country', 'ano']).size().reset_index(name='counts')
# Ordene os dados pelo ano
df_agrupado_cole = df_agrupado_cole.sort_values(by='counts')
df_agrupado = df_agrupado.sort_values(by='ano')

# Crie o gráfico de barras empilhadas
fig = px.bar(df_agrupado_cole, color='country', y='counts', x='year', title='Contagem de Coletas por País (Empilhado por Ano)',
             labels={'counts': 'Contagem de Coletas', 'country': 'País', 'year': 'Ano'},
             barmode='stack')
fig2 = px.bar(df_agrupado, color='country', y='counts', x='ano', title='Contagem de Coletas por País (Empilhado por Ano)',
             labels={'counts': 'Contagem de Coletas', 'country': 'País', 'year': 'Ano'},
             barmode='stack')

barras_paises_dict['ano'] = fig2
barras_paises_dict['year'] = fig


In [36]:

barras_sem_brasil_dict = {}
df_ano = df[df['country'] != 'Brasil']
df_agrupado_cole = df_ano.groupby(['country', 'year']).size().reset_index(name='counts')
df_agrupado = df_ano.groupby(['country', 'ano']).size().reset_index(name='counts')
# Ordene os dados pelo ano
df_agrupado_cole = df_agrupado_cole.sort_values(by='counts')
df_agrupado = df_agrupado.sort_values(by='ano')

# Crie o gráfico de barras empilhadas
fig = px.bar(df_agrupado_cole, color='country', y='counts', x='year', title='Contagem de Coletas por País (Empilhado por Ano)',
             labels={'counts': 'Contagem de Coletas', 'country': 'País', 'year': 'Ano'},
             barmode='stack')
fig2 = px.bar(df_agrupado, color='country', y='counts', x='ano', title='Contagem de Coletas por País (Empilhado por Ano)',
             labels={'counts': 'Contagem de Coletas', 'country': 'País', 'year': 'Ano'},
             barmode='stack')

barras_sem_brasil_dict['ano'] = fig2
barras_sem_brasil_dict['year'] = fig
fig

ValueError: Mime type rendering requires nbformat>=4.2.0 but it is not installed

Figure({
    'data': [{'alignmentgroup': 'True',
              'hovertemplate': 'País=Alemanha<br>Ano=%{x}<br>Contagem de Coletas=%{y}<extra></extra>',
              'legendgroup': 'Alemanha',
              'marker': {'color': '#636efa', 'pattern': {'shape': ''}},
              'name': 'Alemanha',
              'offsetgroup': 'Alemanha',
              'orientation': 'v',
              'showlegend': True,
              'textposition': 'auto',
              'type': 'bar',
              'x': array(['10', '2022', '2007', '2009', '2006'], dtype=object),
              'xaxis': 'x',
              'y': array([ 1,  1,  4,  5, 29], dtype=int64),
              'yaxis': 'y'},
             {'alignmentgroup': 'True',
              'hovertemplate': 'País=Nova Zelândia<br>Ano=%{x}<br>Contagem de Coletas=%{y}<extra></extra>',
              'legendgroup': 'Nova Zelândia',
              'marker': {'color': '#EF553B', 'pattern': {'shape': ''}},
              'name': 'Nova Zelândia',
              'offsetgroup': 'Nova Zelândia',
              'orientation': 'v',
              'showlegend': True,
              'textposition': 'auto',
              'type': 'bar',
              'x': array(['2024', '2023', '2017', '2013', '2016', '2022'], dtype=object),
              'xaxis': 'x',
              'y': array([ 1,  1,  1,  2,  3, 57], dtype=int64),
              'yaxis': 'y'},
             {'alignmentgroup': 'True',
              'hovertemplate': 'País=Equador<br>Ano=%{x}<br>Contagem de Coletas=%{y}<extra></extra>',
              'legendgroup': 'Equador',
              'marker': {'color': '#00cc96', 'pattern': {'shape': ''}},
              'name': 'Equador',
              'offsetgroup': 'Equador',
              'orientation': 'v',
              'showlegend': True,
              'textposition': 'auto',
              'type': 'bar',
              'x': array(['2009'], dtype=object),
              'xaxis': 'x',
              'y': array([1], dtype=int64),
              'yaxis': 'y'},
             {'alignmentgroup': 'True',
              'hovertemplate': 'País=Chile<br>Ano=%{x}<br>Contagem de Coletas=%{y}<extra></extra>',
              'legendgroup': 'Chile',
              'marker': {'color': '#ab63fa', 'pattern': {'shape': ''}},
              'name': 'Chile',
              'offsetgroup': 'Chile',
              'orientation': 'v',
              'showlegend': True,
              'textposition': 'auto',
              'type': 'bar',
              'x': array(['2018', '2002', '2017', '2016', '2007', '2010'], dtype=object),
              'xaxis': 'x',
              'y': array([ 1,  3,  7, 13, 16, 19], dtype=int64),
              'yaxis': 'y'},
             {'alignmentgroup': 'True',
              'hovertemplate': 'País=Honduras<br>Ano=%{x}<br>Contagem de Coletas=%{y}<extra></extra>',
              'legendgroup': 'Honduras',
              'marker': {'color': '#FFA15A', 'pattern': {'shape': ''}},
              'name': 'Honduras',
              'offsetgroup': 'Honduras',
              'orientation': 'v',
              'showlegend': True,
              'textposition': 'auto',
              'type': 'bar',
              'x': array(['2010'], dtype=object),
              'xaxis': 'x',
              'y': array([2], dtype=int64),
              'yaxis': 'y'},
             {'alignmentgroup': 'True',
              'hovertemplate': 'País=Guiana Francesa<br>Ano=%{x}<br>Contagem de Coletas=%{y}<extra></extra>',
              'legendgroup': 'Guiana Francesa',
              'marker': {'color': '#19d3f3', 'pattern': {'shape': ''}},
              'name': 'Guiana Francesa',
              'offsetgroup': 'Guiana Francesa',
              'orientation': 'v',
              'showlegend': True,
              'textposition': 'auto',
              'type': 'bar',
              'x': array(['2008'], dtype=object),
              'xaxis': 'x',
              'y': array([2], dtype=int64),
              'yaxis': 'y'},
             {'alignmentgroup': 'True',
              'hovertemplate': 'País=Estados Uni

In [19]:
df_ano = df.copy()

df_ano = df_ano[df_ano['country'] == 'Brasil']
df_agrupado = df_ano.groupby(['stateProvince', 'ano']).size().reset_index(name='counts')

# Ordene os dados pelo ano
df_agrupado = df_agrupado.sort_values(by='counts')

# Crie o gráfico de barras empilhadas
fig = px.bar(df_agrupado, color='stateProvince', y='counts', x='ano', title='Contagem de Coletas por País (Empilhado por Ano)',
             labels={'counts': 'Contagem de Coletas', 'country': 'País', 'year': 'Ano'},
             barmode='stack')

fig


ValueError: Mime type rendering requires nbformat>=4.2.0 but it is not installed

Figure({
    'data': [{'alignmentgroup': 'True',
              'hovertemplate': 'stateProvince=Bahia<br>ano=%{x}<br>Contagem de Coletas=%{y}<extra></extra>',
              'legendgroup': 'Bahia',
              'marker': {'color': '#636efa', 'pattern': {'shape': ''}},
              'name': 'Bahia',
              'offsetgroup': 'Bahia',
              'orientation': 'v',
              'showlegend': True,
              'textposition': 'auto',
              'type': 'bar',
              'x': array([2022, 2021, 2014, 2023, 2013, 2019, 2016, 2017, 2020], dtype=int64),
              'xaxis': 'x',
              'y': array([  1,   5,  10,  12,  12,  22,  26,  83, 122], dtype=int64),
              'yaxis': 'y'},
             {'alignmentgroup': 'True',
              'hovertemplate': ('stateProvince=Rio Grande do Su' ... 'de Coletas=%{y}<extra></extra>'),
              'legendgroup': 'Rio Grande do Sul',
              'marker': {'color': '#EF553B', 'pattern': {'shape': ''}},
              'name': 'Rio Grande do Sul',
              'offsetgroup': 'Rio Grande do Sul',
              'orientation': 'v',
              'showlegend': True,
              'textposition': 'auto',
              'type': 'bar',
              'x': array([2023, 2020, 2013, 2014, 2016, 2012, 2017], dtype=int64),
              'xaxis': 'x',
              'y': array([ 1,  5,  6, 12, 19, 34, 35], dtype=int64),
              'yaxis': 'y'},
             {'alignmentgroup': 'True',
              'hovertemplate': ('stateProvince=Rio Grande do No' ... 'de Coletas=%{y}<extra></extra>'),
              'legendgroup': 'Rio Grande do Norte',
              'marker': {'color': '#00cc96', 'pattern': {'shape': ''}},
              'name': 'Rio Grande do Norte',
              'offsetgroup': 'Rio Grande do Norte',
              'orientation': 'v',
              'showlegend': True,
              'textposition': 'auto',
              'type': 'bar',
              'x': array([2018, 2020, 2019, 2017], dtype=int64),
              'xaxis': 'x',
              'y': array([ 1,  3,  4, 44], dtype=int64),
              'yaxis': 'y'},
             {'alignmentgroup': 'True',
              'hovertemplate': 'stateProvince=Pernambuco<br>ano=%{x}<br>Contagem de Coletas=%{y}<extra></extra>',
              'legendgroup': 'Pernambuco',
              'marker': {'color': '#ab63fa', 'pattern': {'shape': ''}},
              'name': 'Pernambuco',
              'offsetgroup': 'Pernambuco',
              'orientation': 'v',
              'showlegend': True,
              'textposition': 'auto',
              'type': 'bar',
              'x': array([2018, 2020, 2012, 2014, 2013, 2017], dtype=int64),
              'xaxis': 'x',
              'y': array([ 1,  6, 12, 15, 36, 43], dtype=int64),
              'yaxis': 'y'},
             {'alignmentgroup': 'True',
              'hovertemplate': 'stateProvince=Pará<br>ano=%{x}<br>Contagem de Coletas=%{y}<extra></extra>',
              'legendgroup': 'Pará',
              'marker': {'color': '#FFA15A', 'pattern': {'shape': ''}},
              'name': 'Pará',
              'offsetgroup': 'Pará',
              'orientation': 'v',
              'showlegend': True,
              'textposition': 'auto',
              'type': 'bar',
              'x': array([2012, 2021, 2018, 2023, 2022, 2016, 2013, 2015, 2014, 2017], dtype=int64),
              'xaxis': 'x',
              'y': array([  1,   4,  12,  17,  19,  70,  76,  84,  85, 118], dtype=int64),
              'yaxis': 'y'},
             {'alignmentgroup': 'True',
              'hovertemplate': 'stateProvince=Paraíba<br>ano=%{x}<br>Contagem de Coletas=%{y}<extra></extra>',
              'legendgroup': 'Paraíba',
              'marker': {'color': '#19d3f3', 'pattern': {'shape': ''}},
              'name': 'Paraíba',
              'offsetgroup': 'Paraíba',
              'orientation': 'v',
              'showlegend': True,
              'textposition': 'auto',
              'type': 'bar',
              'x': array([2023, 2013, 2019, 202

In [20]:
df_ano = df.copy()

df_ano = df_ano[df_ano['country'] == 'Brasil']
df_ano = df_ano[df_ano['stateProvince'] != 'Minas Gerais']
df_agrupado = df_ano.groupby(['stateProvince', 'ano']).size().reset_index(name='counts')

# Ordene os dados pelo ano
df_agrupado = df_agrupado.sort_values(by='counts')

# Crie o gráfico de barras empilhadas
fig = px.bar(df_agrupado, color='stateProvince', y='counts', x='ano', title='Contagem de Coletas por País (Empilhado por Ano)',
             labels={'counts': 'Contagem de Coletas', 'country': 'País', 'year': 'Ano'},
             barmode='stack')

fig

ValueError: Mime type rendering requires nbformat>=4.2.0 but it is not installed

Figure({
    'data': [{'alignmentgroup': 'True',
              'hovertemplate': 'stateProvince=Bahia<br>ano=%{x}<br>Contagem de Coletas=%{y}<extra></extra>',
              'legendgroup': 'Bahia',
              'marker': {'color': '#636efa', 'pattern': {'shape': ''}},
              'name': 'Bahia',
              'offsetgroup': 'Bahia',
              'orientation': 'v',
              'showlegend': True,
              'textposition': 'auto',
              'type': 'bar',
              'x': array([2022, 2021, 2014, 2013, 2023, 2019, 2016, 2017, 2020], dtype=int64),
              'xaxis': 'x',
              'y': array([  1,   5,  10,  12,  12,  22,  26,  83, 122], dtype=int64),
              'yaxis': 'y'},
             {'alignmentgroup': 'True',
              'hovertemplate': 'stateProvince=Pernambuco<br>ano=%{x}<br>Contagem de Coletas=%{y}<extra></extra>',
              'legendgroup': 'Pernambuco',
              'marker': {'color': '#EF553B', 'pattern': {'shape': ''}},
              'name': 'Pernambuco',
              'offsetgroup': 'Pernambuco',
              'orientation': 'v',
              'showlegend': True,
              'textposition': 'auto',
              'type': 'bar',
              'x': array([2018, 2020, 2012, 2014, 2013, 2017], dtype=int64),
              'xaxis': 'x',
              'y': array([ 1,  6, 12, 15, 36, 43], dtype=int64),
              'yaxis': 'y'},
             {'alignmentgroup': 'True',
              'hovertemplate': ('stateProvince=Espírito Santo<b' ... 'de Coletas=%{y}<extra></extra>'),
              'legendgroup': 'Espírito Santo',
              'marker': {'color': '#00cc96', 'pattern': {'shape': ''}},
              'name': 'Espírito Santo',
              'offsetgroup': 'Espírito Santo',
              'orientation': 'v',
              'showlegend': True,
              'textposition': 'auto',
              'type': 'bar',
              'x': array([2018, 2021, 2014, 2016, 2023, 2013, 2020, 2017, 2019, 2022], dtype=int64),
              'xaxis': 'x',
              'y': array([  1,   1,   4,  11,  12,  15,  37,  52, 105, 130], dtype=int64),
              'yaxis': 'y'},
             {'alignmentgroup': 'True',
              'hovertemplate': ('stateProvince=Mato Grosso do S' ... 'de Coletas=%{y}<extra></extra>'),
              'legendgroup': 'Mato Grosso do Sul',
              'marker': {'color': '#ab63fa', 'pattern': {'shape': ''}},
              'name': 'Mato Grosso do Sul',
              'offsetgroup': 'Mato Grosso do Sul',
              'orientation': 'v',
              'showlegend': True,
              'textposition': 'auto',
              'type': 'bar',
              'x': array([2018, 2023, 2014], dtype=int64),
              'xaxis': 'x',
              'y': array([ 1,  6, 44], dtype=int64),
              'yaxis': 'y'},
             {'alignmentgroup': 'True',
              'hovertemplate': ('stateProvince=Rio Grande do No' ... 'de Coletas=%{y}<extra></extra>'),
              'legendgroup': 'Rio Grande do Norte',
              'marker': {'color': '#FFA15A', 'pattern': {'shape': ''}},
              'name': 'Rio Grande do Norte',
              'offsetgroup': 'Rio Grande do Norte',
              'orientation': 'v',
              'showlegend': True,
              'textposition': 'auto',
              'type': 'bar',
              'x': array([2018, 2020, 2019, 2017], dtype=int64),
              'xaxis': 'x',
              'y': array([ 1,  3,  4, 44], dtype=int64),
              'yaxis': 'y'},
             {'alignmentgroup': 'True',
              'hovertemplate': 'stateProvince=Paraíba<br>ano=%{x}<br>Contagem de Coletas=%{y}<extra></extra>',
              'legendgroup': 'Paraíba',
              'marker': {'color': '#19d3f3', 'pattern': {'shape': ''}},
              'name': 'Paraíba',
              'offsetgroup': 'Paraíba',
              'orientation': 'v',
              'showlegend': True,
              'textposition': 'auto',
              'type': 'bar',
              'x': array([2013, 2023, 2019, 2022, 2017], d

In [ ]:
from folium import Marker, Map, LayerControl, Icon, FeatureGroup
from folium.plugins import HeatMap
from random import randint
from datetime import date

# Verificar a coluna decimal para ser inteiro
df_mapa = df.copy()

def fazer_mapa(df_mapa):
    family = 'family'  # Certifique-se de definir a família correta aqui
    zoom_level = 4
    genero_cores = {}
    cores = ['red','blue','green','purple','orange','darkred','lightred','beige','darkblue','darkgreen','cadetblue','darkpurple','white','pink','lightblue','lightgreen','gray','black','lightgray']

    # Lista com o maior e menor valor de latitude e longitude
    latitude_values = [df_mapa['decimalLatitude'].max(), df_mapa['decimalLatitude'].min()]
    longitude_values = [df_mapa['decimalLongitude'].max(), df_mapa['decimalLongitude'].min()]

    if all(value is None for value in latitude_values) and all(value is None for value in longitude_values):
        latitude_values = [-19.8688655, -19.8688655]
        longitude_values = [-43.9695513, -43.9695513]
        zoom_level = 16

    # Média para determinar o meio do mapa
    latitude_media = sum(latitude_values) / 2
    longitude_media = sum(longitude_values) / 2

    # Inicializa o mapa com o meio calculado e o nível de zoom
    mapa_family = Map(location=[latitude_media, longitude_media], zoom_start=zoom_level, name='Map')

    # Prepara dados para o HeatMap
    heat_data = [[float(row['decimalLatitude']), float(row['decimalLongitude'])] for index, row in df_mapa.iterrows() if not pd.isnull(row['decimalLatitude']) and not pd.isnull(row['decimalLongitude'])]

    for index, colecao in df_mapa.iterrows():
        if not pd.isnull(colecao['decimalLatitude']) and not pd.isnull(colecao['decimalLongitude']):
            latitude = colecao['decimalLatitude']
            longitude = colecao['decimalLongitude']

            # Associa uma cor a um gênero e cria uma camada de mapa para cada gênero
            if colecao['genus'] not in genero_cores and len(cores) > 0:
                cor = cores.pop(0)
                genero_cores[colecao['genus']] = {'camada': FeatureGroup(name='<span style="color: '+cor+';">\u25A0 </span>'+colecao['genus']), 'cor': cor}
            elif len(cores) <= 0:
                cor = "#{:06x}".format(randint(0, 0xFFFFFF))
                genero_cores[colecao['genus']] = {'camada': FeatureGroup(name='<span style="color: '+cor+';">\u25A0 </span>'+colecao['genus']), 'cor': cor}
            else:
                cor = genero_cores[colecao['genus']]['cor']

            # Prepara informações para o popup do marcador no mapa
            texto_popup = f"UFMG-AC{colecao['catalogNumber']}\n{colecao['country']} ({colecao['countryCode']}), {colecao['stateProvince']},{colecao['county']},\n {latitude};{longitude},\n{colecao['eventDate']}, Col.: , Cod.: {colecao['associatedSequences']}"

            # Adiciona marcadores para cada coleção ao mapa
            Marker([latitude, longitude], popup=texto_popup, tooltip=colecao['catalogNumber'], icon=Icon(color=cor), name='marker').add_to(genero_cores[colecao['genus']]['camada'])

        # Adiciona as camadas de mapa para cada gênero ao mapa principal
        for genero in sorted(genero_cores.keys()):
            camada = genero_cores[genero]['camada']
            mapa_family.add_child(camada)

    # Adiciona um HeatMap ao mapa
    HeatMap(heat_data, name='HeatMap', show=False).add_to(mapa_family)

    # Adiciona um controle de camadas ao mapa
    LayerControl().add_to(mapa_family)

    # Retorna a representação HTML do mapa
    mapa_html = mapa_family._repr_html_()
    return mapa_html

a = fazer_mapa(df_mapa)
a


In [34]:
# análise de parte todo para verificar as especies de cada coletor


In [23]:
''' 
análise multivariável (catalogNumber, recordedBy, individualcount, sex, lifeStage, preparations, yea, mounth, samplingProtocol, continent,
                       country, StateProvince, country, order, superfamily, family, genus, specie, identifiedBy,dataIdentified
)
'''
df_cp = df[['order', 'superfamily', 'family', 'genus', 'specie',]]#'recordedBy', 'individualCount', 'sex', 'lifeStage', 'preparations', 'samplingProtocol', 'country', 'stateProvince', 'country','identifiedBy','dateIdentified']]
# fig = px.parallel_categories(df)
# fig


In [24]:
# sazonalidade com o mês e dia da semana


In [37]:
# from dash_extensions import Plotly
# Inicializar o aplicativo JupyterDash
app = Dash(__name__)

# Layout do aplicativo
app.layout = html.Div([
    dcc.Graph(id='treemap_brasil', figure=treemap_class_taxon),
    dcc.Dropdown(
        id='tax-selector',
        options=[{'label': tax.capitalize(), 'value': tax} for tax in tax_levels],
        value='order'
    ),
    dcc.Dropdown(
        id='time-selector',
        options=[{'label': 'Identificação', 'value': 'ano'}, {'label': 'Coleta', 'value': 'year'}],
        value='ano',
    ),
    dcc.Graph(id='graph'),
    dcc.Graph(id='graph1'),
    dcc.Dropdown(id='line-tax-dropdown'),
    dcc.Graph(id='line-graph'),
    dcc.Graph(id='unique_tax', figure=grafico_unicos_taxon),
    dcc.Graph(id='percent_null', figure=grafico_valores_null),
    dcc.Graph(id='percent_null_denpendent', figure=grafico_valores_null_dependente),
    dcc.Graph(id='coletores'),
    dcc.Graph(id='equipe'),

    dcc.Graph(id='treemap_paises',figure=treemap_paises),
    html.Div(id='output'),
    dcc.Graph(id='barras-country',clear_on_unhover=True),
    dcc.Graph(
        id='mapa',
        # figure=Plotly(),
        config={'displayModeBar': False}
    )
])

line_dropdown_value = 'all'
# @app.callback(
#     Output('mapa', 'figure'),
#     [Input('tax-selector', 'value'),]
# )
# def update_map(tax_selector):
#     # Chame sua função fazer_mapa() aqui para obter o HTML do mapa e renderizá-lo no Dash
#     mapa_html = fazer_mapa()
#     return {'data': [], 'layout': {'template': mapa_html}}
@app.callback(
    [Output('line-tax-dropdown', 'options'),
     Output('line-tax-dropdown', 'value')],
    [Input('tax-selector', 'value')]
)
def update_line_tax_dropdown(selected_tax):
    global line_dropdown_value
    # Obter as opções de itens disponíveis no gráfico de linha para o nível taxonômico selecionado
    line_tax_dropdown_options = [{'label': f'Mostrar todas as {selected_tax}', 'value': nome} if nome =='all' else {'label': nome, 'value': nome} for nome in linha_tax_dict[selected_tax]]

    # Definir o valor inicial do dropdown para a seleção armazenada
    initial_value = 'all'
    
    return line_tax_dropdown_options, initial_value

@app.callback(
    [Output('graph', 'figure'),
     Output('graph1', 'figure'),
     Output('line-graph', 'figure'),
     Output('coletores', 'figure'),
     Output('equipe', 'figure'),
     Output('barras-country', 'figure'),
     Output('output', 'children')],
    [Input('tax-selector', 'value'),
     Input('time-selector', 'value'),
     Input('line-tax-dropdown', 'value'),
     Input('barras-country', 'hoverData'),
     Input('barras-country', 'clickData')]  
)
def update_figure(tax_selection, time_selection, line_tax_selection, hoverData, clickData):
    global current_tax_selection, current_time_selection, line_dropdown_value
    current_tax_selection = tax_selection
    current_time_selection = time_selection

    # Atualize o gráfico de linha apenas se a seleção mudou
    if line_tax_selection != line_dropdown_value:
        line_dropdown_value = line_tax_selection

    # Access the appropriate area_empilhada_dict entry and retrieve the figure
    output_text = f'Selecionado: {line_tax_selection} - {clickData} {hoverData}'
    figure_to_return = area_empilhada_dict[tax_selection][time_selection]
    histograma_return = histograma_tax_dict[tax_selection]
    line_figure = linha_tax_dict[tax_selection][line_tax_selection] if line_tax_selection in linha_tax_dict[tax_selection] else linha_tax_dict[tax_selection]
    barras_coletores_return = barras_coletores_dict[time_selection]
    barras_equipe_return = barras_equipe_dict[time_selection]
    barras_paises_return = barras_paises_dict[time_selection]

    if hoverData and 'points' in hoverData:
        hovered_country = hoverData['points'][0]['curveNumber']
        for trace in barras_paises_return.data:
            trace['marker']['opacity'] = 0.5
        barras_paises_return.data[hovered_country]['marker']['opacity'] = 1.0
    else:
        for trace in barras_paises_return.data:
            trace['marker']['opacity'] = 1.0

    return figure_to_return, histograma_return, line_figure, barras_coletores_return, barras_equipe_return, barras_paises_return, output_text

app.run_server(debug=True, port=8051)
